# Mask R-CNN on videos Using pretrained model

## Install Matterport Mask-RCNN in Google Colab

In [ ]:
# https://stackoverflow.com/questions/53740577/does-any-one-got-attributeerror-str-object-has-no-attribute-decode-whi
!pip3 install 'h5py==2.10.0' --force-reinstall

In [ ]:
#enforce tensorflow version 1 
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN
%cd Mask_RCNN

In [ ]:
# https://github.com/matterport/Mask_RCNN/issues/1754#issuecomment-776493501

with open('mrcnn/model.py') as f:
    model_file = f.read()

with open('mrcnn/model.py', 'w') as f:
    model_file = model_file.replace("self.keras_model = self.build(mode=mode, config=config)",
                                    "self.keras_model = self.build(mode=mode, config=config)\n        self.keras_model.metrics_tensors = []")
    f.write(model_file)

In [ ]:
!pip3 install -r requirements.txt
!python3 setup.py install

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

## Imports

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append("samples/coco/")  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = "images"

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
import colorsys
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

colors = random_colors(len(class_names))
color_map = dict(zip(class_names, np.array(colors)))

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config);

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Run Object Detection

In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

In [ ]:
"""
Mask R-CNN
Display and Visualization Functions.

Copyright (c) 2017 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla
"""

import os
import sys
import random
import itertools
import colorsys

import numpy as np
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import IPython.display

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils


############################################################
#  Visualization
############################################################

def display_images(images, titles=None, cols=4, cmap=None, norm=None,
                   interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of image tensors in HWC format.
    titles: optional. A list of titles to display with each image.
    cols: number of images per row
    cmap: Optional. Color map to use. For example, "Blues".
    norm: Optional. A Normalize instance to map values to colors.
    interpolation: Optional. Image interpolation to use for display.
    """
    titles = titles if titles is not None else [""] * len(images)
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        i += 1
    plt.show()


def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image


# obtaining a black and white image where white represents the objects detected and black is the background
def apply_mask_white(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] + color[c] * 255,
                                  image[:, :, c])
    return image

# detecting separate instances of all the objects in the image and assigning the same colour to the same object across different frames but identifying the object
def display_instances_objects(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=False,
                      colors=None, captions=None):
  
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # Generate random colors
    colors = colors or random_colors(N)

    masked_image_full = image.astype(np.uint32).copy()
    masks_new=[]

    for i in range(N):
        color = (1,1,1)

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            # masked_image_full= apply_mask(masked_image_full, mask, color)
            masked_image = np.zeros(image.shape, dtype = "uint8") 
            masked_image = apply_mask_white(masked_image, mask, color).clip(0, 255).astype("uint8")
            masked_new=grayScaleImg(image, masked_image )
            masks_new.append(masked_new)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)

    return masks_new

        
def display_instances_black(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=False,
                      colors=None, captions=None):
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image =np.zeros(image.shape, dtype = "uint8")
    for i in range(N):
        color = (1,1,1)

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)



        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask_white(masked_image, mask, color)
           # masked_image= masked_image.clip(0, 255).astype("uint8")
        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 2)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=(0.5,0.5, 0.5))
            ax.add_patch(p)
    ax.imshow(masked_image.clip(0, 255).astype("uint8"))

    if auto_show:
        plt.show()
    return masked_image.clip(0, 255).astype("uint8")

def display_instances(image, boxes, masks, class_ids, class_names,color_map,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=False,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """

    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    c= np.unique(np.array(class_ids))
    colors = colors or random_colors(len(c))
    dctnry = dict(zip(c, np.array(colors)))
    print(c)
    print(dctnry)
    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = color_map.get(class_names[class_ids[i]]) 

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)



        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)

    ax.imshow(masked_image.astype(np.uint8))

    if auto_show:
        plt.show()





def display_top_masks(image, mask, class_ids, class_names, limit=4):
    """Display the given image and the top few class masks."""
    to_display = []
    titles = []
    to_display.append(image)
    titles.append("H x W={}x{}".format(image.shape[0], image.shape[1]))
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]])
                 for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),
                                    key=lambda r: r[1], reverse=True) if v[1] > 0]
    # Generate images and titles
    for i in range(limit):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] + 1), -1)
        to_display.append(m)
        titles.append(class_names[class_id] if class_id != -1 else "-")
    display_images(to_display, titles=titles, cols=limit + 1, cmap="Blues_r")

 

In [ ]:
import cv2
image = skimage.io.imread('/content/bed1.jpg')

results = model.detect([image], verbose=0)

# Visualize results
r = results[0]
display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, color_map,r['scores'], show_bbox=False)


In [ ]:
black= display_instances_black(image, r['rois'], r['masks'], r['class_ids'],
                               class_names, r['scores'], show_bbox=False )

In [ ]:
import cv2
# Create structuring element, dilate and bitwise-and
def grayScaleImg(image, black):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.dilate(black, kernel, iterations=3)
    result = cv2.bitwise_and(image, dilate)
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    # Enhance image contrast
    gray = cv2.equalizeHist(gray)
    plt.imshow(gray, cmap='gray')
    return gray

In [ ]:
g=grayScaleImg(image, black)

In [ ]:
def get_iou(bb1, bb2):

    y11, x11, y12, x12 = bb1
    y21, x21, y22, x22 = bb2
    assert x11 < x12
    assert y11< y12 
    assert x21 < x22 
    assert y21< y22 

    # determine the coordinates of the intersection rectangle
    x_left = max(x11, x21 )
    y_top = max(y11, y21 )
    x_right = min(x12, x22)
    y_bottom = min(y12, y22)

    if x_right < x_left or y_bottom < y_top:
        return 0.0
    print(x_right < x_left)
    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (x12 - x11 ) * (y12 - y11)
    bb2_area = ( x22  - x21) * (y22 - y21)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
display_instances_objects(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], show_bbox=False)

In [ ]:
display_images([display_instances_objects(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], show_bbox=False)][0], cmap= 'gray')

In [ ]:
m=display_instances_objects(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], show_bbox=False)

In [ ]:
plt.imshow(m[1].astype(np.uint8), cmap= 'gray')

In [ ]:
def image_resize(images, rois, desired_size=32):
        res=[]
        for i in range(0, len(images)):
                im = images[i]
                k= r['rois'][i]
                im= m[i][k[0]:k[2], k[1]:k[3]]
                old_size = im.shape[:2] # old_size is in (height, width) format

                ratio = float(desired_size)/max(old_size)
                new_size = tuple([int(x*ratio) for x in old_size])

                # new_size should be in (width, height) format

                im = cv2.resize(im, (new_size[1], new_size[0]))

                delta_w = desired_size - new_size[1]
                delta_h = desired_size - new_size[0]
                top, bottom = delta_h//2, delta_h-(delta_h//2)
                left, right = delta_w//2, delta_w-(delta_w//2)

                color = [0, 0, 0]
                new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
                    value=color)
                res.append(new_im)
        return res
              

In [ ]:
os.makedirs("home_imagesn")
for i in range(0, len(m)): 
  plt.imshow(m[i], cmap= 'gray')
  plt.axis('off')
  cv2.imwrite("home_imagesn/object"+str(i)+".png", m[i])

In [ ]:
cropped= image_resize(m, r['rois'])
os.makedirs("home_images_croppednew1")
for i in range(0, len(m)): 
  plt.imshow(cropped[i], cmap= 'gray')
  print(cropped[i].shape)
  plt.axis('off')
  #plt.savefig("home_images_croppedn/object"+str(i)+".png", bbox_inches='tight',pad_inches = 0)
  cv2.imwrite("home_images_croppednew1/object"+str(i)+".png", cropped[i])

In [ ]:
from PIL import Image
image = Image.open("/content/bed1.jpg")
image = image.resize((3072,3072),Image.ANTIALIAS)
image.save(fp="newimage.png")

Generating Images and Videos from the implemented code: 

In [ ]:
!zip -r imgsLocn.zip home_images1/

In [ ]:
!zip -r imgsCroppednew.zip home_images_croppednew1/

In [ ]:
display_top_masks(image, r['masks'], r['class_ids'], class_names)

In [ ]:
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

In [ ]:
def get_color_dict():
  all_colors = visualize.random_colors(100)
  color_dict = {}
  i = 0
  for c in class_names:
    if not c in color_dict:
      color_dict[c] = all_colors[i]
      i = i+1
  color_dict["background"] = (0,0,0)
  return color_dict

COLOR_MAP = get_color_dict()

In [ ]:
def label_to_color_image(labels):
  # Adds color defined by the dataset colormap to the label.
  h,w = labels.shape
  img = np.zeros([h,w,3])
  img = np.zeros((h,w),dtype=(float,3))
  for i in range(h):
    for j in range(w):
      img[i][j] = np.array(COLOR_MAP[CLASS_NAMES_MASKRCNN[labels[i][j]]])        
  img = img*255
  return img.astype(np.uint8)

def combine_masks(img, result):
  boxes = result['rois']
  masks = result['masks']
  class_ids = result['class_ids']
  
  N = boxes.shape[0]
  h,w,c = img.shape
  seg_map = np.zeros((h,w))
  for i in range(N):
    mask = masks[:, :, i]
    mask = mask.astype(np.uint8)
    seg_map = seg_map + mask*class_ids[i]
  
  return seg_map.astype(np.uint8)

def merge_images(foreground, background, alpha=0.3):
  out_img = np.zeros(background.shape,dtype=background.dtype)
  out_img[:,:,:] = (alpha * background[:,:,:]) + ((1-alpha) * foreground[:,:,:])
  return out_img

def get_masked_image1(image, result):
  """
  Applies masks from the results to the given image
  
  """
  boxes = result['rois']
  masks = result['masks']
  
  N = boxes.shape[0]
  if not N:
    print("\n*** No instances to display *** \n")

  colors = visualize.random_colors(N)
  masked_image =  masked_image =np.zeros(image.shape, dtype = "uint8")


  for i in range(N):
      color = (1,1,1)

      # Mask
      mask = masks[:, :, i]
      
      masked_image = visualize.apply_mask(masked_image, mask, color)
  return masked_image.astype(np.uint8)

class Object:
  def __init__(self, rois, color):
    self.rois = rois
    self.color = color

def get_color(result, prevFrame):
  colors=[]
  temp_colors=random_colors(len(result['class_ids']))
  for i in range(len(result['class_ids'])):
    curr= result['class_ids'][i]
    color= temp_colors[i]
    if curr in prevFrame:
          if len(prevFrame[curr])!=0:
            iou=0
            for obj in prevFrame[curr]:
              rois=obj.rois
              temp= get_iou(rois, result['rois'][i])
              if temp>=0.3 and temp>iou:
                color= obj.color
    colors.append(color)  
    print("clrs",colors)  
  return colors


def get_masked_image_maintainColor_GRAY(image, result, prevFrame, flag):
  boxes = result['rois']
  masks = result['masks']
  
  N = boxes.shape[0]
  if not N:
    print("\n*** No instances to display *** \n")

  colors = get_color(result, prevFrame)
  masked_image = image.astype(np.uint32).copy()
  print("ccc",colors)
  keys = result['class_ids']
  # newFrame ={key: [] for key in keys}
  newFrame= prevFrame
  for i in range(N):
      color = colors[i]
      if result['class_ids'][i] in newFrame:
          newFrame[result['class_ids'][i]].append(Object(result['rois'][i], color))      
      else:
         newFrame[result['class_ids'][i]]= [Object(result['rois'][i], color)]
      # Mask
      mask = masks[:, :, i]
      #masked_image.clip(0, 255).astype("uint8")
    
  black= display_instances_black(image, result['rois'], result['masks'], result['class_ids'], 
                                class_names, result['scores'], show_bbox=False)
  if flag==1:    
     return newFrame, black.astype(np.uint8)
      
  masked=grayScaleImg(image, black)
  return newFrame, masked.astype(np.uint8)
      
 


def get_masked_image_maintainColor(image, result, prevFrame):
  """
  Applies masks from the results to the given image

  first, check if the prevFrame contains the same object in the current frame ['key':[<result, color>]]

  there can be several objects of the same class 

  for each object of the same class calculate the iou

  assign the same colour to the curr object

  update the prevFrame by creating a new frame dictionary containing each object's result and color 
  
  """
  boxes = result['rois']
  masks = result['masks']
  
  N = boxes.shape[0]
  if not N:
    print("\n*** No instances to display *** \n")

  colors = get_color(result, prevFrame)
  masked_image = image.astype(np.uint32).copy()
  print("ccc",colors)
  keys = result['class_ids']
  # newFrame ={key: [] for key in keys}
  newFrame= prevFrame
  for i in range(N):
      color = colors[i]
      if result['class_ids'][i] in newFrame:
          newFrame[result['class_ids'][i]].append(Object(result['rois'][i], color))      
      else:
         newFrame[result['class_ids'][i]]= [Object(result['rois'][i], color)]
      # Mask
      mask = masks[:, :, i]
      masked_image = visualize.apply_mask(masked_image, mask, color)
  return newFrame, masked_image.astype(np.uint8)

def get_masked_image(image, result):
  """
  Applies masks from the results to the given image
  
  """
  boxes = result['rois']
  masks = result['masks']
  
  N = boxes.shape[0]
  if not N:
    print("\n*** No instances to display *** \n")

  colors = visualize.random_colors(N)
  masked_image = image.astype(np.uint32).copy()

  for i in range(N):
      color = colors[i]

      # Mask
      mask = masks[:, :, i]
      masked_image = visualize.apply_mask(masked_image, mask, color)
  return masked_image.astype(np.uint8)


def get_coloured_mask(image, boxes, masks, class_ids, class_names,color_map,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=False,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """

    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    c= np.unique(np.array(class_ids))
    colors = colors or random_colors(len(c))
    dctnry = dict(zip(c, np.array(colors)))
    print(c)
    print(dctnry)
    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = color_map.get(class_names[class_ids[i]]) 

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)



        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    return masked_image.astype(np.uint8)
 


def print_fps(video):
  # Find OpenCV version
  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

  if int(major_ver)  < 3 :
      fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
      print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
  else :
      fps = video.get(cv2.CAP_PROP_FPS)
      print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))


def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [ ]:
import os
VIDEO_DIR = os.path.join("", "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "bathroom_contrast")

try:
    if not os.path.exists(VIDEO_SAVE_DIR):
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print ('Error: Creating directory of data')

os.chdir('./videos')

from google.colab import files
uploads = files.upload()

os.chdir('../')

In [ ]:
import cv2

# number of images to be processed at once
batch_size = 1
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = batch_size

config = InferenceConfig()
config.display()

model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config
)
model.load_weights(COCO_MODEL_PATH, by_name=True)
CLASS_NAMES_MASKRCNN = ['background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                 'bus', 'train', 'truck', 'boat', 'traffic light',
                 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
for upload in uploads.keys():
  capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, upload))
  
  print_fps(capture)
  
  try:
      if not os.path.exists(VIDEO_SAVE_DIR):
          os.makedirs(VIDEO_SAVE_DIR)
  except OSError:
      print ('Error: Creating directory of data')
  frames = []
  frame_count = 0
  keys = CLASS_NAMES_MASKRCNN
  prevFrameD ={key: [] for key in keys}
  while True:
      ret, frame = capture.read()
      # Bail out when the video file ends
      if not ret:
          break
          
      # Save each frame of the video to a list
      frame_count += 1
      frames.append(frame)
      print('frame_count :{0}'.format(frame_count))
      if len(frames) == batch_size:
          results = model.detect(frames, verbose=0)
          print('Predicted')
          for i, item in enumerate(zip(frames, results)):
              frame = item[0]
              r = item[1]
              #seg_map = combine_masks(frame, r)
              #seg_image = label_to_color_image(seg_map)
              #frame = merge_images(seg_image, frame)
              #frame= get_coloured_mask(frame,r['rois'], r['masks'], r['class_ids'], 
              #                        class_names, color_map,r['scores'], show_bbox=False)
              #frame = get_masked_image(frame, r)

              ##maintains colour of object instances:
              #prevFrameD, frame= get_masked_image_maintainColor(frame, r,  prevFrameD)

              ##get greyScale object instead of the mask and makes it greyscale :)
              prevFrameD, frame= get_masked_image_maintainColor_GRAY(frame, r,  prevFrameD, 0)

              # prevFrameD, frame= get_masked_image_maintainColor_GRAY(frame, r,  prevFrameD, 0)



              name = '{0}.jpg'.format(frame_count + i - batch_size)
              name = os.path.join(VIDEO_SAVE_DIR, name)
              cv2.imwrite(name, frame)
          # Clear the frames array to start the next batch
          frames = []

  capture.release()

In [ ]:
import glob
import os

images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "bathroom-mask.mp4")
make_video(outvid, images, fps=20)